In [112]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from numpy.linalg import svd
from collections import Counter
from scipy.special import softmax

In [22]:
def bitarray(num):
    return np.array(map(int,'{:016b}'.format(num)))

def fill_uniform_random(d):
    # create a D-dimensional hypercube in the range (-1,1) in each axis
    # and place one sample randomly in each bin
    samples = 2**d
    coords = random.rand(samples,d)
    sign_lists= []
    for r in range(d-1,-1,-1):
        sign_lists.append(([0] * 2**r + [1] * 2**r) * 2**(d-r-1))
    
    sign_arr = np.array(sign_lists).T*2-1
    return coords * sign_arr, sign_arr


def orthant_detector_layer(d):
    # create a D-dimensional hypercube in the range (-1,1) in each axis
    # and place one sample randomly in each bin
    sign_lists= []
    for r in range(d-1,-1,-1):
        sign_lists.append(([0] * 2**r + [1] * 2**r) * 2**(d-r-1))
    
    sign_arr = np.array(sign_lists).T*2-1
    return sign_arr



In [169]:
def random_transform_matrix(d):
    u,w,_ = svd(random.rand(d,d))
    return np.matmul(u,np.diag(w)), np.matmul(np.diag(0.2/w),u.T)

In [16]:
a,b=random_transform_matrix(3)
np.matmul(a,np.diag(b))

array([[-0.55198919, -0.58055321, -0.00444318],
       [-1.20311205,  0.15031833,  0.07831523],
       [-0.9448312 ,  0.14776111, -0.09712783]])

In [20]:
a, a_inv = random_transform_matrix(4)
print(a)
print(a_inv)
print(np.matmul(a,a_inv))
print(np.matmul(a_inv,a))

[[-0.70897872 -0.44856906  0.0735389  -0.07829513]
 [-1.52806388  0.20799945 -0.31018044 -0.00583245]
 [-0.9282658   0.66084181  0.23792004 -0.01576273]
 [-1.43657828 -0.42688059  0.13990534  0.05502933]]
[[-0.12302113 -0.2651478  -0.16107156 -0.24927333]
 [-0.51952776  0.24090269  0.76537974 -0.49440842]
 [ 0.4136062  -1.7445535   1.33813803  0.78687214]
 [-8.29324153 -0.61778936 -1.66963332  5.82886226]]
[[ 1.00000000e+00 -2.08166817e-17  1.38777878e-16 -2.77555756e-16]
 [-4.85722573e-17  1.00000000e+00 -3.20923843e-16 -4.85722573e-17]
 [ 8.32667268e-17 -3.76434994e-16  1.00000000e+00 -4.16333634e-17]
 [-1.66533454e-16 -5.55111512e-17  1.38777878e-17  1.00000000e+00]]
[[ 1.00000000e+00  2.77555756e-17 -4.16333634e-17  3.46944695e-18]
 [ 2.22044605e-16  1.00000000e+00  1.52655666e-16  1.38777878e-17]
 [-1.55431223e-15  8.32667268e-16  1.00000000e+00 -4.16333634e-17]
 [ 1.77635684e-15  0.00000000e+00 -8.88178420e-16  1.00000000e+00]]


In [ ]:
fill_uniform_random(5)

In [ ]:
samples,layer2 = fill_uniform_random(5);
#np.tanh(samples*5)

In [ ]:
samples.shape, layer2.shape

In [ ]:
np.argmax(np.matmul(np.tanh(samples*5),layer2.T),axis=1)

In [ ]:
layer2

In [206]:
def create_class_assignments(n_samples, n_class):
    class_assignments = (random.rand(n_samples-n_class)*8).astype(int)
    class_assignments = np.hstack([class_assignments,np.arange(n_class,dtype=int)])
    random.shuffle(class_assignments)
    return class_assignments
    
def generate_layers_and_samples(n_dims, n_class):
    random.seed(1234)
    a, a_inv = random_transform_matrix(n_dims)
    b = orthant_detector(n_dims)
    n_samples = 2**n_dims
    class_assignments = create_class_assignments(n_samples,n_class)    
    c = class_assignment_layer(class_assignments)
    c = (1/np.sum(c,axis=1)[:,np.newaxis])*c
    samples = random.rand(n_samples, n_dims)
    samples = samples*b
    samples = np.arctanh(samples)
    samples = np.matmul(a_inv, samples.T)
    return a, b*0.5, c, class_assignments, samples
    
def class_assignment_layer(class_assignments):
    # create a layer that produces the maximum value for the correct class
    n_classes = max(class_assignments)+1
    n_samples = len(class_assignments)
    l = np.tile(class_assignments,(n_classes,1)).astype(np.float)
    for i in range(n_classes):
        l[i,:] = l[i,:]== i
        
    return l

In [207]:
a, b, c, class_assignments, samples = generate_layers_and_samples(6,6)

In [208]:

a.shape, b.shape, c.shape, class_assignments.shape, samples.shape

((6, 6), (64, 6), (8, 64), (64,), (6, 64))

In [209]:
net = lambda s: softmax(np.matmul(c,np.tanh(np.matmul(b, np.tanh(np.matmul(a,s))))))

In [210]:
s = samples[:,4:7]

In [211]:
class_assignments

array([0, 0, 5, 0, 0, 1, 5, 5, 3, 7, 3, 7, 4, 0, 4, 7, 2, 4, 3, 5, 4, 7,
       6, 5, 3, 4, 1, 0, 1, 0, 3, 7, 7, 3, 2, 3, 4, 1, 7, 7, 0, 1, 5, 1,
       4, 4, 5, 4, 4, 0, 1, 2, 4, 4, 6, 4, 6, 3, 0, 7, 0, 2, 3, 4])

In [212]:
np.argmax(net(samples),axis=0)==class_assignments

array([False, False,  True, False, False, False,  True,  True,  True,
       False,  True,  True, False,  True, False, False, False, False,
       False,  True, False, False,  True,  True,  True, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False,  True,
        True, False, False, False, False, False, False, False, False,
        True, False,  True, False, False, False, False, False,  True,
       False])

In [254]:
k = 3
a1 = np.tanh(500*np.matmul(a,samples[:,k]))
print(a1)
a2 = np.tanh(np.matmul(0.1*b,a1)-0.15)
print("\na2")
print(np.array2string(a2,precision=3, suppress_small=True))

a3 = np.matmul(c,a2)
print("\na3")
print(np.array2string(a3,precision=2, suppress_small=True))
print()
print(class_assignments[k])

[-1.         -1.         -1.         -0.89083423  1.          1.        ]

a2
[-0.055  0.045  0.045  0.144 -0.144 -0.045 -0.045  0.055 -0.154 -0.055
 -0.055  0.045 -0.24  -0.144 -0.144 -0.045 -0.154 -0.055 -0.055  0.045
 -0.24  -0.144 -0.144 -0.045 -0.25  -0.154 -0.154 -0.055 -0.332 -0.24
 -0.24  -0.144 -0.154 -0.055 -0.055  0.045 -0.24  -0.144 -0.144 -0.045
 -0.25  -0.154 -0.154 -0.055 -0.332 -0.24  -0.24  -0.144 -0.25  -0.154
 -0.154 -0.055 -0.332 -0.24  -0.24  -0.144 -0.341 -0.25  -0.25  -0.154
 -0.417 -0.332 -0.332 -0.24 ]

a3
[-0.14 -0.15 -0.15 -0.15 -0.21 -0.05 -0.24 -0.09]

0


In [248]:
print(np.dot(c[0,:],a2))

0.00742774533213985


In [239]:
print(np.array2string(c,precision=2, suppress_small=True))

[[0.09 0.09 0.   0.09 0.09 0.   0.   0.   0.   0.   0.   0.   0.   0.09
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.09
  0.   0.09 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.09 0.
  0.   0.   0.   0.   0.   0.   0.   0.09 0.   0.   0.   0.   0.   0.
  0.   0.   0.09 0.   0.09 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.14 0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.14 0.
  0.14 0.   0.   0.   0.   0.   0.   0.   0.   0.14 0.   0.   0.   0.14
  0.   0.14 0.   0.   0.   0.   0.   0.   0.14 0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0. 

In [176]:
class_assignments[:5]

array([5, 1, 7, 3, 3])

In [177]:
c[7,:]

array([0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.])

In [189]:
sum(ct[3,:])

1.0000000000000002

In [184]:
ct = (1/np.sum(c,axis=1)[:,np.newaxis])*c

In [161]:

c/np.sum(c,axis=0)

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1

In [168]:
np.matmul(np.diag(1/np.sum(c,axis=1)),c)

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.08333333, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.08333333, 0.        ,
        0.08333333, 0.08333333, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.08333333, 0.        ,
        0.        , 0.08333333, 0.        , 0.08333333, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.08333333, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.08333333,
        0.        , 0.        , 0.08333333, 0.08333333, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.08333333, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0. 